In [1]:
#TODOs 
# Add Data Origin (Linkedin in this case)

In [2]:
from urllib.request import urlopen as ureq
import urllib

In [3]:
import pandas as pd
import re
import random
from datetime import date, timedelta

import selenium
from bs4 import BeautifulSoup as soup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from warnings import warn

In [4]:
from bs4 import BeautifulSoup as soup

In [5]:
#url = 'https://www.seek.com.au/data-analyst-jobs/in-All-Sydney-NSW?daterange=1&sortmode=ListedDate'
url = 'https://www.seek.com.au/data-analyst-junior-jobs/in-sydney?daterange=2'

### Opening up connection and getting HTML info from the webpage

In [6]:
driver = webdriver.Chrome()
driver.get(url)

In [7]:
driver.find_element_by_xpath('//span[@id="SearchSummary"]').text

'102 jobs found'

In [8]:
new_jobs_posted = re.findall('\d+',driver.find_element_by_xpath('//span[@id="SearchSummary"]').text)[0]
new_jobs_posted

'102'

In [9]:
#Instatiating the lists to fill when looping
job_titles=[]
company_name=[]
job_location=[]
date_scraped=[]
links=[]
job_industry=[]
job_desc=[]
job_type=[]
job_seniority=[]
page = 0
links2= []

#Check for the app to stop if the job was already scrapped
df = pd.read_csv('Seek_data_info.csv')

In [10]:
#Contenedores1
contenedores = driver.find_elements_by_xpath('//article[@data-automation="premiumJob"]')

In [11]:
#Contenedores2
contenedores2 = driver.find_elements_by_xpath('//article[@data-automation="normalJob"]')

In [12]:
while 1>0:
#Jobs en class = "premiumJob"
    links = []
    sleep(random.uniform(3.2,6.8))
    contenedores = driver.find_elements_by_xpath('//article[@data-automation="premiumJob"]')
    contenedores2 = driver.find_elements_by_xpath('//article[@data-automation="normalJob"]')
    for job_post in contenedores:
        try:
            job_area = job_post.find_element_by_xpath('.//a[@data-automation="jobArea"]').text
        except: 
            job_area = ''
        # Job Title
        job_titles.append(job_post.find_element_by_xpath('.//a[@class="_2S5REPk"]').text)
        # Company name
        try:
            company_name.append(job_post.find_element_by_xpath('.//a[@data-automation="jobCompany"]').text)
        except:
            company_name.append('Private Advertiser')
        # Job Location
        job_location.append(job_post.find_element_by_xpath('.//a[@data-automation="jobLocation"]').text + ' ' + job_area)
        #Date Scraped
        date_scraped.append(str(date.today()))
        #Links
        links.append(job_post.find_element_by_xpath('.//a[@class="_2S5REPk"]').get_attribute('href'))
        links2.append(job_post.find_element_by_xpath('.//a[@class="_2S5REPk"]').get_attribute('href'))

    #Jobs en class = "normalJob"
    for job_post in contenedores2:
        try:
            job_area = job_post.find_element_by_xpath('.//a[@data-automation="jobArea"]').text
        except: 
            job_area = ''
        # Job Title
        job_titles.append(job_post.find_element_by_xpath('.//a[@class="_2S5REPk"]').text)
        # Company name
        try:
            company_name.append(job_post.find_element_by_xpath('.//a[@data-automation="jobCompany"]').text)
        except:
            company_name.append('Private Advertiser')
        # Job Location
        job_location.append(job_post.find_element_by_xpath('.//a[@data-automation="jobLocation"]').text + ' ' + job_area)
        #Date Scraped
        date_scraped.append(str(date.today()))
        #Links
        links.append(job_post.find_element_by_xpath('.//a[@class="_2S5REPk"]').get_attribute('href'))
        links2.append(job_post.find_element_by_xpath('.//a[@class="_2S5REPk"]').get_attribute('href'))

    for i in range(0,len(links)):
        sleep(random.uniform(1.5,4.8))
        url= links[i]
        uclient = ureq(url)
        page_html = uclient.read()
        uclient.close()
        page_soup = soup(page_html, 'html.parser')
        if len(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})) == 4:
            job_industry.append(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})[1].text.strip())
            job_desc.append(page_soup.findAll('div',{'data-automation':'jobAdDetails'})[0].text.strip())
            job_type.append(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})[3].text.strip())
            job_seniority.append('NA')
        
        elif len(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})) == 5 and (re.search(r'\d+',page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})[3].text.strip()) is None):
            job_industry.append(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})[2].text.strip())
            job_desc.append(page_soup.findAll('div',{'data-automation':'jobAdDetails'})[0].text.strip())
            job_type.append(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})[4].text.strip())
            job_seniority.append('NA')
        
        elif len(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})) == 5 and (re.search(r'\d+',page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})[3].text.strip()) is not None):
            job_industry.append(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})[1].text.strip())
            job_desc.append(page_soup.findAll('div',{'data-automation':'jobAdDetails'})[0].text.strip())
            job_type.append(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})[4].text.strip())
            job_seniority.append('NA')
        
        elif len(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})) == 6:
            job_industry.append(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})[2].text.strip())
            job_desc.append(page_soup.findAll('div',{'data-automation':'jobAdDetails'})[0].text.strip())
            job_type.append(page_soup.findAll('div',{'class':'yvsb870 _14uh9942g _1lOnv_4'})[5].text.strip())
            job_seniority.append('NA')
    
    try:
        driver.find_element_by_xpath('//a[@class="_24YOjgT"]').click()
        page += 1
    except:
        break

In [13]:
dataframe_structure = {'Job Title': job_titles, 'Company Name':company_name, 'Location':job_location, 'Date Scraped':date_scraped, 'Job URL':links2, 'Job Industry':job_industry, 'Job Type':job_type, 'Job Seniority':job_seniority, 'Job Description':job_desc}

In [14]:
print('Job titles ----')
print(len(job_titles))    
print(job_titles[0:2])
print('Job company ----')
print(len(company_name))
print(company_name[0:2])
print('Job Location ----')
print(len(job_location))
print(job_location[0:2])
print('Date scraped ----')
print(len(date_scraped))
print(date_scraped[0:2])
print('Job links ----')
print(len(links2))
print(links2[0:2])
print('Job type ----')
print(len(job_type))
print(job_type[0:2])
print('Job industry ----')
print(job_industry[0:2])
print('Job seniority ----')
print(job_seniority[0:2])

Job titles ----
102
['Junior Data Analyst', 'Junior Data Analyst']
Job company ----
102
['TABCORP', 'Guzman y Gomez']
Job Location ----
102
['Sydney CBD, Inner West & Eastern Suburbs', 'Sydney CBD, Inner West & Eastern Suburbs']
Date scraped ----
102
['2021-08-27', '2021-08-27']
Job links ----
102
['https://www.seek.com.au/job/53618692?type=standout#searchRequestToken=337a891f-3bd0-4669-864c-839d6997604f', 'https://www.seek.com.au/job/53636737?type=standout#searchRequestToken=337a891f-3bd0-4669-864c-839d6997604f']
Job type ----
102
['Full Time', 'Full Time']
Job industry ----
['Banking & Financial Services', 'Information & Communication Technology']
Job seniority ----
['NA', 'NA']


In [15]:
df = pd.read_csv('Seek_data_info.csv', engine='python')
last_index_in_csv = int(df.iloc[-1]['Unnamed: 0'])
last_index_in_csv_1 = last_index_in_csv+1
df = pd.DataFrame(dataframe_structure)
df.index += last_index_in_csv_1

In [16]:
df.head()

,Job Title,Company Name,Location,Date Scraped,Job URL,Job Industry,Job Type,Job Seniority,Job Description
1239,Junior Data Analyst,TABCORP,"Sydney CBD, Inner West & Eastern Suburbs",2021-08-27,https://www.seek.com.au/job/53618692?type=stan...,Banking & Financial Services,Full Time,NA,"One day you’re helping us boldly innovate, ano..."
1240,Junior Data Analyst,Guzman y Gomez,"Sydney CBD, Inner West & Eastern Suburbs",2021-08-27,https://www.seek.com.au/job/53636737?type=stan...,Information & Communication Technology,Full Time,NA,Guzman y Gomez is the fastest growing fast foo...
1241,Junior Business Analyst,Brilliance Financial Technology,"Sydney CBD, Inner West & Eastern Suburbs",2021-08-27,https://www.seek.com.au/job/53666044?type=stan...,Banking & Financial Services,Full Time,NA,About the RoleOur company is a leading provide...
1242,Quantitative Data Analyst,Peoplefusion,Sydney,2021-08-27,https://www.seek.com.au/job/53667899?type=stan...,Banking & Financial Services,Full Time,NA,Genuine opportunity for a graduate or early ca...
1243,Data Analyst,Mercer,Sydney,2021-08-27,https://www.seek.com.au/job/53619448?type=stan...,Human Resources & Recruitment,Full Time,NA,Company:MercerDescription:Junior Project Manag...


In [17]:
df.to_csv('Seek_data_info.csv',mode='a',header=False)